In [1]:
import numpy as np
import MDAnalysis as md
import pickle
import os
from shapeGMM import gmm_shapes

In [2]:
# reorder cluster numbers based on populations in descending order
def reorder_gmm_cluster_obj(sgmm_obj):
    # determine metadata based on clusters
    #n_frames = sgmm_obj.n_frames
    n_frames = sgmm_obj.n_frames
    cluster_ids, cluster_populations = np.unique(sgmm_obj.clusters,return_counts=True)
    n_clusters = cluster_ids.size
    print("Number of clusters:", n_clusters)
    print("Populations prior to reorder:", cluster_populations/n_frames)
    # determine sort key
    sort_key = np.argsort(cluster_populations)[::-1]
    sorted_cluster_ids = cluster_ids[sort_key]
    new_clusters = np.empty(n_frames,dtype=int)
    for frame in range(n_frames):
        new_clusters[frame] = np.argwhere(sorted_cluster_ids == sgmm_obj.clusters[frame])
    cluster_ids, cluster_populations = np.unique(new_clusters,return_counts=True)
    print("Populations after reorder:", cluster_populations/n_frames)
    # repopulate object
    sgmm_obj.centers = sgmm_obj.centers[sort_key]
    sgmm_obj.precisions = sgmm_obj.precisions[sort_key]
    sgmm_obj.weights = sgmm_obj.weights[sort_key]
    sgmm_obj.ln_weights = sgmm_obj.ln_weights[sort_key]
    sgmm_obj.lpdets = sgmm_obj.lpdets[sort_key]
    sgmm_obj.clusters = new_clusters

# Read trajectory and save backbone atom selections

In [19]:
# read trajectory data
data_path = '../../../pnas2012-2f4k-360K-protein/'
backbone_selection_101 = "(name C and resid 42) or (name C CA N and not resid 42 76) or (name N and resid 76)"
# LOAD DATA
prmtopFileName =  data_path + 'pnas2012-2f4k-360K-protein.pdb'
trajFiles = [data_path + files for files in sorted(os.listdir(data_path)) if files.endswith('.dcd')]
coord = md.Universe(prmtopFileName,trajFiles)
sel_backbone_101 = coord.select_atoms(backbone_selection_101)
print("Number of atoms in trajectory:", coord.atoms.n_atoms)
print("Number of frames in trajectory:",coord.trajectory.n_frames)
print("Number of atoms being analyzed:",sel_backbone_101.n_atoms)
print("Number of frames being analyzed:",coord.trajectory.n_frames)
traj_backbone_101 = np.empty((coord.trajectory.n_frames,sel_backbone_101.n_atoms,3),dtype=float)
count = 0
for ts in coord.trajectory:
    traj_backbone_101[count,:,:] = sel_backbone_101.positions - sel_backbone_101.center_of_geometry()
    count += 1

Number of atoms in trajectory: 577
Number of frames in trajectory: 1526041
Number of atoms being analyzed: 101
Number of frames being analyzed: 1526041


# Read clustering from cluster index and position file and create new object

In [2]:
file_name = "6_clusters_opt_weighted_backbone_gmm.clusters"
clusters_wsgmm_6_state = np.loadtxt(file_name).astype(int)

In [3]:
file_name = "6_clusters_opt_weighted_backbone_gmm.positions"
positions_wsgmm_6_state = np.loadtxt(file_name).reshape((61042, 101, 3))

In [6]:
wsgmm = gmm_shapes.ShapeGMM(n_clusters=6,init_cluster_method="read",verbose=True)
wsgmm.fit_weighted(positions_wsgmm_6_state,clusters=clusters_wsgmm_6_state)

Number of frames being analyzed: 61042
Number of particles being analyzed: 101
Number of dimensions (must be 3): 3
Initializing clustering using method: read
Weights from initial clusters in fit_weighted: [0.33801317 0.21296812 0.13341634 0.10913797 0.10414141 0.10232299]
0 [0.3382606  0.21281237 0.13326996 0.10902342 0.10399044 0.1026432 ] 23184297.7409202
1 [0.33849565 0.212617   0.13317133 0.10897976 0.10387184 0.10286443] 23184461.614208233
2 [0.33864931 0.21248184 0.13313629 0.10895637 0.10377688 0.10299931] 23184489.948992006
3 [0.33875468 0.21238734 0.13312755 0.1089447  0.10369875 0.10308698] 23184506.372845013
4 [0.33882775 0.21231882 0.13312716 0.10894154 0.10363721 0.10314752] 23184512.81002284
5 [0.33887938 0.21226919 0.13313068 0.10894277 0.10358816 0.10318981] 23184516.6097561
6 [0.33891665 0.21223282 0.13313703 0.10894631 0.10354818 0.10321901] 23184519.485328596
7 [0.338944   0.21220585 0.13314686 0.10895122 0.10351327 0.1032388 ] 23184521.5009237
8 [0.33896436 0.212185

array([[[ -3.60868617,  -6.14156554,  12.61132335],
        [ -2.89689964,  -5.64993151,  11.56464156],
        [ -3.23111294,  -4.2549911 ,  10.90551131],
        ...,
        [ 17.66899029,  -8.15979362, -24.87537003],
        [ 17.47081765,  -7.07032393, -23.81565953],
        [ 17.1680789 ,  -7.48902154, -22.55053108]],

       [[ 14.42119675,  -1.14661912,  -2.09491545],
        [ 14.69508181,  -1.97086531,  -3.07270845],
        [ 13.66339606,  -2.83112306,  -3.56343946],
        ...,
        [ -5.46795351,   4.9373367 ,   4.74879003],
        [ -5.40944378,   4.06510261,   5.97689729],
        [ -4.15343207,   3.78525979,   6.4835511 ]],

       [[  9.48843094, -12.10434667,  -3.77977411],
        [  9.56794013, -10.86711923,  -3.3440766 ],
        [  9.73347296, -10.51783659,  -1.94760224],
        ...,
        [ -1.43468972,   9.07022557,   2.61510113],
        [ -0.82012129,   9.97115605,   1.51638409],
        [ -0.33994241,   9.46340575,   0.35965984]],

       ...,

      

In [15]:
reorder_gmm_cluster_obj(wsgmm)

Number of clusters: 6
Populations prior to reorder: [0.33801317 0.21300088 0.13382589 0.10936732 0.10242128 0.10337145]
Populations after reorder: [0.33801317 0.21300088 0.13382589 0.10936732 0.10337145 0.10242128]


# Save new object

In [16]:
file_name = "hp35_wsgmm_6_clusters_reorderd.pickle"
pickle_file = open(file_name,"wb")
pickle.dump(wsgmm,pickle_file)
pickle_file.close()

# Read object and predict clustering on entire trajectory

In [3]:
file_name = "hp35_wsgmm_6_clusters_reorderd.pickle"
pickle_file = open(file_name,"rb")
wsgmm_loaded = pickle.load(pickle_file)
pickle_file.close()

In [21]:
full_traj_clusters, full_traj_coordinates, full_traj_log_lik = wsgmm_loaded.predict_weighted(traj_backbone_101)

In [25]:
print("Predict log likelihood per frame:",full_traj_log_lik/traj_backbone_101.shape[0])
print("Train log likelihood per frame  :",wsgmm_loaded.log_likelihood/wsgmm_loaded.n_frames)

Predict log likelihood per frame: 379.20946939236495
Train log likelihood per frame  : 379.8134702799295


In [22]:
print(np.unique(full_traj_clusters,return_counts=True)[1])

[513969 326358 203663 167295 157838 156918]


In [23]:
np.savetxt("hp35_wsgmm_6_clusters_reorderd_full_traj.clusters",full_traj_clusters)

## Let's get some analysis done quickly

In [6]:
print(wsgmm_loaded)